# NIH grants dataset creation

### Through NIH API, data is mined for cross-referencing between winning applications and grant requests in this notebook

In [92]:
import numpy as np
import pandas as pd
import requests
import pickle
import json
import csv
import time
import re
from tqdm import tqdm
# importing the libraries
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
tqdm.pandas()
from ast import literal_eval
import matplotlib.pyplot as plt
import Levenshtein
import io

In [7]:
foas = pd.read_excel('nih_r01_funding_expired.xlsx')

In [8]:
len(foas.Document_Number.tolist())

4456

In this notebook we will mine only the following research program grants: R01, R03, R21, R33 , R34:
[This link](https://www.niaid.nih.gov/grants-contracts/research-project-grants) explains thoroughly the difference between R01 and R03/R21.<br>

<u>Note:</u><br>
FOA $\rightarrow$ Fund Opportunity Announcement

## Mine R01 grants
R01 is the NIH standard independent research project grant. An R01 is meant to give you four or five years of support to complete a project, publish, and reapply before the grant ends.

An R01 is for mature research projects that are hypothesis-driven with strong preliminary data. R01s provide up to five years of support, with a budget that reflects the costs required to complete the project. 

In [25]:
j=0
for foa in tqdm(foas.Document_Number.tolist()[j::]):
    if j%500==0 and j>0:
        time.sleep(300)
    headers = {
        'accept': 'application/json',
    }

    json_data = {
        "criteria": {
            "foa": [
                foa,
            ]
        },
        "limit": 500,
        "sort_field": "appl_id",
        "sort_order": "desc"
     }
#     time.sleep(np.random.randint(2,5))
    time.sleep(2)
    response = requests.post('https://api.reporter.nih.gov/v2/projects/search', headers=headers, json=json_data)
    data = json.loads(response.content)
    df = df.append(pd.json_normalize(data['results'])).reset_index(drop=True)
    j+=1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 821/821 [8:33:18<00:00, 37.51s/it]


In [29]:
df.to_csv('nih_winning_grants_R01.csv',index=False)

## Mine R03, R021, R33,R34 grants
Small (**R03**) and exploratory/developmental research (**R21**) grants are designed for investigators who have small defined projects where preliminary data are not required.

If researcher's project is not ready for an **R01**, he then can consider a two-year small grant (**R03**) or exploratory/developmental research grant (**R21**) to generate preliminary data or develop approaches that could support a hypothesis that can be tested under the **R01** activity code, later on.

The NIH Planning Grant Program (**R34**) provides support for the initial development of a clinical trial or research project. This program may support:

* establishment of the research team,
* development of tools for data management and research oversight
* development of a trial design or experimental research designs 
* finalization of the protocol
* preparation of an operations/procedures manual
* pilot studies or collection of feasibility data for subsequent research projects*

The **R33** award is to provide a second phase for the support for innovative exploratory and development research activities initiated under the **R21** mechanism. - this is why all the **R33** requests will be later dropped.


In [ ]:
foas = pd.read_excel('nih_R03R21R33R34_funding_expired.xlsx')

In [ ]:
df = pd.read_csv('nih_winning_data_R03R21R33R34.csv')
foa = foas.Document_Number.tolist()
range_start = 0 # if breaks at some number, change to the number where the mining stopped
for i in tqdm(range(range_start,3535,5)):
    if i%500==0 and i>0:
        time.sleep(600)
        
    headers = {
        'accept': 'application/json',
    }

    json_data = {
        "criteria": {
            "foa": foa[i:i+5]
        },
        "limit": 500,
        "sort_field": "appl_id",
        "sort_order": "desc"
     }
    time.sleep(2)
    response = requests.post('https://api.reporter.nih.gov/v2/projects/search', headers=headers, json=json_data)
    data = json.loads(response.content)
    df = df.append(pd.json_normalize(data['results'])).reset_index(drop=True)
    df.to_csv('nih_winning_data_R03R21R33R34.csv',index=False)

In [ ]:
df_r01 = pd.read_csv('nih_winning_grants_R01.csv')
df_rest = pd.read_csv('nih_winning_data_R03R21R33R34.csv')

In [36]:
df_final = pd.concat([df_r01,df_rest]).reset_index(drop=True)
df_final = df_final.drop_duplicates(subset='appl_id')

In [37]:
df_final.to_csv('nih_winning_data_ALL.csv',index=False)

Drop R33 grants:

In [ ]:
df_final = pd.read_csv('nih_winning_data_ALL.csv')

In [38]:
foas = pd.read_excel('nih_R03R21R33R34_funding_expired.xlsx')

In [40]:
r21= foas['Activity_Code'].str.contains('R21',regex=False)
r01 = foas['Activity_Code'].str.contains('R01',regex=False)
r03 = foas['Activity_Code'].str.contains('R03',regex=False)
r34 = foas['Activity_Code'].str.contains('R34',regex=False)

In [41]:
foas.shape

(3539, 11)

In [42]:
foas =foas.loc[r21 | r01 | r03 | r34]
foas.shape

(3394, 11)

In [43]:
foas = foas[~foas.Document_Number.str.contains('NOT',case=True,regex=False)].reset_index(drop=True)

In [44]:
foas.shape

(3390, 11)

Create unified grant calls file and filtered winning applications file

In [45]:
foas_r01 = pd.read_excel('nih_r01_funding_expiredxlsx.xlsx')
foas_r01.shape

(4456, 11)

In [47]:
foas_final = pd.concat([foas_r01,foas]).reset_index(drop=True)

In [55]:
foas_final.to_excel('foas_final.xlsx',index=False)

In [50]:
df_final = df_final[df_final['full_foa'].isin(foas_final['Document_Number'])].reset_index(drop=True)

In [51]:
df_final.to_csv('nih_winning_data_filtered.csv',index=False)

In [53]:
df_final.shape

(226994, 80)

## Get grant calls information

In [56]:
df = pd.read_csv('nih_winning_data_filtered.csv',low_memory=False)
foas = pd.read_excel('foas_final.xlsx')

In [57]:
RFA = foas[foas['Document_Number'].str.contains('RFA',regex=False)]
PAR= foas[foas['Document_Number'].str.contains('PAR',regex=False)]
PA =foas[foas['Document_Number'].str.contains('PA-',regex=False)]
PAS =foas[foas['Document_Number'].str.contains('PAS',regex=False)]

### Create a function to scrape short text description out of nih grant calls

In [63]:
def case_1(soup):
    a = soup.find(attrs = {"class":'row'})
    if a:
        start = "Funding Opportunity Purpose"
        end = "Key Dates"
        abstract= a.text[a.text.find(start)+len(start)+1:a.text.find(end)].replace('\n', '').strip()\
        .replace('\r', '').strip()\
        .replace('\t',' ').strip()\
        .replace('        ',' ').strip()\
        .replace('\xa0','').strip()\
        .replace('--','-').strip()\
        .replace('   ',' ').strip()\
        .replace('  ',' ').strip()
        abstract = abstract.replace('  ',' ').strip()
        return abstract
    return False

def case_2(soup):
    if soup.text.find("PA NUMBER:") >0 and soup.text.find("\n\n\n\n\n\n\n\r\n\r\n")>0:
        return case_panumber(soup)
    
    if case_6(soup):
        return case_6(soup)
    if soup.text.find("PURPOSE OF THIS PA") > 0 or soup.text.find("PURPOSE OF THE PA") >0 or soup.text.find("PURPOSE OF THIS PAR") >0 or soup.text.find("PURPOSE OF THE PAR") >0 or soup.text.find("PURPOSE OF THIS PAS")>0:
        return case_7(soup)
    start = soup.text.find("PURPOSE")
    if start > 0 and (soup.text.find("PURPOSE OF THIS RFA") >0 or soup.text.find("PURPOSE OF THE RFA") >0):
        return False #GO TO CASE 3
    if start > 0 and soup.text.find("Executive Summary") > 0:
        return False #GO TO CASE 4
    if start >0:
        i=0
        c = soup.text[start+len("PURPOSE")::][i]
        while c==' ' or c=='\n' or c=='\r' or c=='\t':
            i+=1
            c=soup.text[start+len("PURPOSE")::][i]
        start_fixed = start+len("PURPOSE")+i
        end1 = soup.text.find('HEALTHY PEOPLE') if soup.text.find('HEALTHY PEOPLE') > 0 else 100000000
        end2 = soup.text.find('RESEARCH OBJECTIVES') if soup.text.find('RESEARCH OBJECTIVES') > 0 else 100000000
        if end1 ==100000000 and end2==100000000 :
            end = -1
        else:
            end = min(end1,end2)     
        if end > 0:
            abstract = soup.text[start_fixed:end].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
            abstract = abstract.replace('  ',' ').strip()
            return abstract
        abstract = soup.text[ start_fixed::].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
        abstract = abstract.replace('  ',' ').strip()
        return abstract
    return False


def case_3(soup):
    start = -1
    if soup.text.find("This is a reissue of RFA MH-03-001.") > 0:
        #specific case
        souptext = soup.text.replace("This is a reissue of RFA MH-03-001.",'')
        start = souptext.find("PURPOSE OF THIS RFA")
        if start >0:
            i=0
            c = souptext[start+len("PURPOSE OF THIS RFA")::][i]
            while c==' ' or c=='\n' or c=='\r' or c=='\t':
                i+=1
                c=souptext[start+len("PURPOSE OF THIS RFA")::][i]
            start_fixed = start+len("PURPOSE OF THIS RFA")+i
            abstract = souptext[start_fixed::].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
                .replace('\n','').strip()\
                .replace('\r', '').strip()\
                .replace('\t',' ').strip()\
                .replace('        ',' ').strip()\
                .replace('\xa0','').strip()\
                .replace('--','-').strip()\
                .replace('   ',' ').strip()\
                .replace('  ',' ').strip()
            abstract = abstract.replace('  ',' ').strip()
            return abstract
    if soup.text.find("PURPOSE OF THIS RFA") > 0:
        start = soup.text.find("PURPOSE OF THIS RFA")
        txt = "PURPOSE OF THIS RFA"
    elif  soup.text.find("PURPOSE OF THE RFA") >0:
        start = soup.text.find("PURPOSE OF THE RFA")
        txt = "PURPOSE OF THE RFA"
    if start >0:
        i=0
        c = soup.text[start+len(txt)::][i]
        while c==' ' or c=='\n' or c=='\r' or c=='\t':
            i+=1
            c=soup.text[start+len(txt)::][i]
        start_fixed = start+len(txt)+i
        end = soup.text.find("RESEARCH OBJECTIVES")
        
        if end >0:
            abstract = soup.text[start_fixed:end].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
            abstract = abstract.replace('  ',' ').strip()
            return abstract
        
        abstract = soup.text[start_fixed::].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
        abstract = abstract.replace('  ',' ').strip()
        return abstract
    
    return False

def case_4(soup):
    start = -1
    if soup.text.find("Executive Summary") > 0:
        start = soup.text.find("Executive Summary")
        txt = "Executive Summary"
    if soup.text.find("Executive\r\n    Summary") > 0:
        start = soup.text.find("Executive\r\n    Summary")
        txt = "Executive\r\n    Summary"
    if start >0:
        i=0
        c = soup.text[start+len(txt)::][i]
        while c==' ' or c=='\n' or c=='\r' or c=='\t':
            i+=1
            c=soup.text[start+len(txt)::][i]
        start_fixed = start+len(txt)+i
    if start> 0:
       if 'Purpose.' in soup.text[start_fixed::] or 'Purpose .' in soup.text[start_fixed::]:
            start_fixed+=len('Purpose. ') if  'Purpose.' in soup.text[start_fixed::] else len('Purpose . ') 
            end1 = soup.text.find("Mechanism\r\n      of Support") if soup.text.find("Mechanism\r\n      of Support") > 0 else 100000000
            end2 = soup.text.find("Mechanism of Support") if soup.text.find("Mechanism of Support") > 0 else 100000000
            end3 = soup.text.find("Mechanism of\r\n      Support") if soup.text.find("Mechanism of\r\n      Support") > 0 else 100000000
            end4 = soup.text.find("Mechanism\r\n     of Support") if soup.text.find("Mechanism\r\n     of Support") > 0 else 100000000


            if end1 ==100000000 and end2==100000000 and end3==100000000 and end4==100000000:
                end = -1
            else:
                end = min(end1,end2,end3,end4)              
            abstract = soup.text[start_fixed:end].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
            abstract = abstract.replace('  ',' ').strip()
            return abstract
    return False

def case_5(soup):
#     Executive\r\n  Summary
    start = -1
    if soup.text.find("Executive Summary") > 0:
        start = soup.text.find("Executive Summary")
        txt = "Executive Summary"
    if soup.text.find("Executive\r\n    Summary") > 0:
        start = soup.text.find("Executive\r\n    Summary")
        txt = "Executive\r\n    Summary"
    if soup.text.find("Executive\r\n  Summary") > 0:
        start = soup.text.find("Executive\r\n  Summary")
        txt = "Executive\r\n  Summary"
    if start >0:
        i=0
        c = soup.text[start+len(txt)::][i]
        while c==' ' or c=='\n' or c=='\r' or c=='\t':
            i+=1
            c=soup.text[start+len(txt)::][i]
        start_fixed = start+len(txt)+i
        end1 = soup.text[start_fixed::].find('.\n') if soup.text[start_fixed::].find('.\n') > 0 else 100000000
        end2 = soup.text[start_fixed::].find('. \n') if soup.text[start_fixed::].find('. \n') > 0 else 100000000
        if end1 ==100000000 and end2==100000000:
            end = -1
        else:
            end = min(end1,end2)
        if end <0:
            return False
        end= start_fixed+end
        abstract = soup.text[start_fixed:end].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
        abstract = abstract.replace('  ',' ').strip()
        return abstract
    return False

def case_6(soup):
    start = -1
    if soup.text.find("Funding Opportunity Purpose") > 0:
        start = soup.text.find("Funding Opportunity Purpose")
        txt = "Funding Opportunity Purpose"
    if  soup.text.find("FOA Purpose") >0:
        start = soup.text.find("FOA Purpose")
        txt = "FOA Purpose"
    if start <0:
        return False
    i=0
    c = soup.text[start+len(txt)::][i]
    while c==' ' or c=='\n' or c=='\r' or c=='\t':
        i+=1
        c=soup.text[start+len(txt)::][i]
    start_fixed = start+len(txt)+i
    end =  soup.text[start_fixed::].find("Key Dates") + start_fixed
    abstract = soup.text[start_fixed:end].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
        .replace('\n','').strip()\
        .replace('\r', '').strip()\
        .replace('\t',' ').strip()\
        .replace('        ',' ').strip()\
        .replace('\xa0','').strip()\
        .replace('--','-').strip()\
        .replace('   ',' ').strip()\
        .replace('  ',' ').strip()
    abstract = abstract.replace('  ',' ').strip()
    return abstract

def case_7(soup):
    start = -1
    if  soup.text.find("PURPOSE OF THIS PAS") >0:
        start = soup.text.find("PURPOSE OF THIS PAS")
        txt = "PURPOSE OF THIS PAS"
    elif soup.text.find("PURPOSE OF THIS PAR") > 0:
        start = soup.text.find("PURPOSE OF THIS PAR")
        txt = "PURPOSE OF THIS PAR"
    elif soup.text.find("PURPOSE OF THIS PA:") > 0:
        start = soup.text.find("PURPOSE OF THIS PA:")
        txt = "PURPOSE OF THIS PA:"
    elif soup.text.find("PURPOSE OF THIS PA") > 0:
        start = soup.text.find("PURPOSE OF THIS PA")
        txt = "PURPOSE OF THIS PA"
    elif  soup.text.find("PURPOSE OF THE PAR") >0:
        start = soup.text.find("PURPOSE OF THE PAR")
        txt = "PURPOSE OF THE PAR"
    elif  soup.text.find("PURPOSE OF THE PA") >0:
        start = soup.text.find("PURPOSE OF THE PA")
        txt = "PURPOSE OF THE PA"

    if start >0:
        i=0
        c = soup.text[start+len(txt)::][i]
        while c==' ' or c=='\n' or c=='\r' or c=='\t':
            i+=1
            c=soup.text[start+len(txt)::][i]
        start_fixed = start+len(txt)+i
        end = soup.text.find("RESEARCH OBJECTIVES")
        
        if end >0:
            abstract = soup.text[start_fixed:end].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
            abstract = abstract.replace('  ',' ').strip()
            return abstract
        
        abstract = soup.text[start_fixed::].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
        abstract = abstract.replace('  ',' ').strip()
        return abstract
    
    return False
def case_panumber(soup):
    start = soup.text.find("PURPOSE")
    if start >0:
        i=0
        c = soup.text[start+len("PURPOSE")::][i]
        while c==' ' or c=='\n' or c=='\r' or c=='\t':
            i+=1
            c=soup.text[start+len("PURPOSE")::][i]
        start_fixed = start+len("PURPOSE")+i
        end1 = soup.text.find('HEALTHY PEOPLE') if soup.text.find('HEALTHY PEOPLE') > 0 else 100000000
        end2 = soup.text.find('RESEARCH OBJECTIVES') if soup.text.find('RESEARCH OBJECTIVES') > 0 else 100000000
        end3 = soup.text.find('ELIGIBILITY REQUIREMENTS') if soup.text.find('ELIGIBILITY REQUIREMENTS') > 0 else 100000000
        if end1 ==100000000 and end2==100000000 and end3==100000000 :
            end = -1
        else:
            end = min(end1,end2,end3)     
        if end > 0:
            abstract = soup.text[start_fixed:end].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
            abstract = abstract.replace('  ',' ').strip()
            return abstract
        abstract = soup.text[ start_fixed::].split('\r\n\r\n')[0].replace('\r\n', ' ').strip()\
            .replace('\n','').strip()\
            .replace('\r', '').strip()\
            .replace('\t',' ').strip()\
            .replace('        ',' ').strip()\
            .replace('\xa0','').strip()\
            .replace('--','-').strip()\
            .replace('   ',' ').strip()\
            .replace('  ',' ').strip()
        abstract = abstract.replace('  ',' ').strip()
        return abstract
    return False
    

In [64]:
def get_descript(url):
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text

    soup = BeautifulSoup(html_content, "lxml")
    if case_1(soup):
        return case_1(soup)
    
    elif case_2(soup):
        return case_2(soup)
    
    elif case_3(soup):
        return case_3(soup)
    
    elif case_4(soup):
        return case_4(soup)
    
    elif case_5(soup):
        return case_5(soup)
    
    elif case_6(soup):
        return case_6(soup)
    else:
        return "Empty"

In [69]:
foas_descrip =[]
k = 0
for url in tqdm(foas['URL'].tolist()):
    if k%2000==0 and k>0:
        time.sleep(300)
    time.sleep(2)
    foas_descrip.append(get_descript(url))
    k+=1
    with open('foas_descript.pickle', 'wb') as handle:
        pickle.dump(foas_descrip, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7846/7846 [6:20:06<00:00,  2.91s/it]


In [72]:
foas['descrip'] = foas_descrip

In [74]:
foas_final = foas.query('descrip!="Empty"').reset_index(drop=True)
print(len(foas_final))

7831


Filter failed scraping

In [77]:
foas_final = foas_final[foas_final.descrip.apply(len) > 60].reset_index(drop=True)
print(len(foas_final))

7781


remove parenthesis from title

In [78]:
foas_final['Title'] = foas_final['Title'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
foas_final['Title'] = foas_final['Title'].apply(lambda x: re.sub(r'\[.*\]','', x))
foas_final['Title'] = foas_final['Title'].apply(lambda x: re.sub(r'\s+$','', x))
foas_final['Title'] = foas_final['Title'].str.replace("\n","")
foas_final['Title'] = foas_final['Title'].str.replace("  "," ")
foas_final['Title'] = foas_final['Title'].str.replace("   "," ")
foas_final = foas_final.drop_duplicates().reset_index(drop=True)
print(len(foas_final))

7126


In [79]:
foas_final.to_csv('NIH_GrantCalls_With_Short_Description.csv',index=False)

In [81]:
foas_final['Document_Number'].unique().shape[0] , foas_final['Title'].unique().shape[0]

(7126, 4410)

In [83]:
df = pd.read_csv('nih_winning_data_filtered.csv',low_memory=False)


,appl_id,subproject_id,fiscal_year,project_num,project_serial_num,award_type,activity_code,award_amount,is_active,principal_investigators,...,full_study_section.srg_code,full_study_section.srg_flex,full_study_section.sra_designator_code,full_study_section.sra_flex_code,full_study_section.group_code,full_study_section.name,full_study_section.url,full_study_section.cmte_id,full_study_section.cluster_irg_code,full_study_section
0,10672784,NaN,2023,1R01HD112031-01,HD112031,1,R01,586134.0,True,"[{'profile_id': 8024549, 'first_name': 'John',...",...,ZHD1,NaN,DSR,H,55,Special Emphasis Panel[ZHD1 DSR-H (55)],NaN,NaN,NaN,NaN
1,10672642,NaN,2023,1R01HD112028-01,HD112028,1,R01,612196.0,True,"[{'profile_id': 8532773, 'first_name': 'James'...",...,ZHD1,NaN,DSR,H,55,Special Emphasis Panel[ZHD1 DSR-H (55)],NaN,NaN,NaN,NaN
2,10670574,NaN,2023,1R01DE032868-01,DE032868,1,R01,553978.0,True,"[{'profile_id': 10451583, 'first_name': 'MARIA...",...,ZDE1,NaN,TO,NaN,12,Special Emphasis Panel[ZDE1 TO (12)],NaN,NaN,NaN,NaN
3,10667157,NaN,2023,1R01AG082142-01,AG082142,1,R01,654613.0,True,"[{'profile_id': 12467458, 'first_name': 'Wang-...",...,ZAG1,NaN,ZIJ,G,J1,Special Emphasis Panel[ZAG1 ZIJ-G (J1)],NaN,NaN,NaN,NaN
4,10667151,NaN,2023,1R01AG082140-01,AG082140,1,R01,622753.0,True,"[{'profile_id': 6721519, 'first_name': 'Mohani...",...,ZAG1,NaN,ZIJ,G,J1,Special Emphasis Panel[ZAG1 ZIJ-G (J1)],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226989,2053349,NaN,1994,3R03AG012003-01S1,AG012003,3,R03,15000.0,False,"[{'profile_id': 1875152, 'first_name': 'REIKO'...",...,ZAG1,NaN,CLL,8,01,ZAG1-CLL-8(01),NaN,NaN,NaN,NaN
226990,2053348,NaN,1993,1R03AG012003-01,AG012003,1,R03,21600.0,False,"[{'profile_id': 1875152, 'first_name': 'REIKO'...",...,ZAG1,NaN,CLL,8,01,ZAG1-CLL-8(01),NaN,NaN,NaN,NaN
226991,2053345,NaN,1994,3R03AG012000-01S1,AG012000,3,R03,18036.0,False,"[{'profile_id': 1950911, 'first_name': 'STEVEN...",...,ZAG1,NaN,CLL,8,01,ZAG1-CLL-8(01),NaN,NaN,NaN,NaN
226992,2053344,NaN,1993,1R03AG012000-01,AG012000,1,R03,27000.0,False,"[{'profile_id': 1950911, 'first_name': 'STEVEN...",...,ZAG1,NaN,CLL,8,01,ZAG1-CLL-8(01),NaN,NaN,NaN,NaN


bind between winners and filtered grant calls

In [85]:
df_final = df[df['full_foa'].isin(foas_final['Document_Number'])].reset_index(drop=True)

In [86]:
df_final = df_final[df_final.contact_pi_name.notnull()].reset_index(drop=True)
foas_final['Document_Number'].unique().shape[0] , foas_final['Title'].unique().shape[0]

(7126, 4410)

In [87]:
df_final.to_csv('NIH_GrantCalls_Winners.csv',index=False)

# Getting PI's publications

get PI's and prepare data for api request

In [88]:
df_final.principal_investigators = df_final.principal_investigators.progress_apply(literal_eval)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225118/225118 [00:08<00:00, 26064.56it/s]


In [92]:
pi_data = []

for index, row in tqdm(df_final.fillna('').iterrows(), total=df_final.shape[0]):
    principal_investigators = row['principal_investigators']
    contact_pi_name = row['contact_pi_name']
    org_name = row['organization.org_name'].lower()
    org_city = row['organization.org_city'].lower()
    
    for pi in principal_investigators:
        pi['contact_pi_name'] = contact_pi_name
        pi['organization.org_name'] = org_name
        pi['organization.org_city'] = org_city
        pi_data.append(pi)

pi_df = pd.DataFrame(pi_data)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225118/225118 [00:12<00:00, 17726.16it/s]


In [94]:
pi_df_only =pi_df[pi_df['is_contact_pi']==True].reset_index(drop=True)
pi_df_only['org_clean'] = pi_df_only['organization.org_name'].str.replace('university','').str.replace('of','').str.replace('the','').str.replace(',','')
pi_df_only = pi_df_only.drop_duplicates().reset_index(drop=True)
pi_df_only.to_csv('pi_df_only_nih.csv',index=False)

Access SCOPUS with pybliometrics

In [116]:
# import pybliometrics.scopus
import pybliometrics.scopus
# pybliometrics.scopus.utils.create_config()
from pybliometrics.scopus import config, ScopusSearch, AbstractRetrieval,AuthorSearch,AuthorRetrieval,SubjectClassifications
from pybliometrics.scopus import AffiliationSearch,AffiliationRetrieval

In [96]:
pi_df_only = pd.merge(pi_df_only,df_final[['contact_pi_name','organization.org_country']].drop_duplicates(),on='contact_pi_name',how = 'left')

## Firstly, get scopus ID

In [4]:
pi_df_only=pd.read_csv('pi_df_only_nih.csv')

In [5]:
df_final=pd.read_csv('NIH_GrantCalls_Winners.csv',low_memory=False)

In [4]:
pi_df_only = pd.merge(pi_df_only,df_final[['contact_pi_name','organization.org_country']].drop_duplicates(),on='contact_pi_name',how = 'left')

In [98]:
pi_df_only = pi_df_only.fillna('')

In [99]:
# remove names with brackets
pi_df_only['first_name'] = pi_df_only['first_name'].apply(lambda x: x.split(' (')[0])
pi_df_only['last_name'] = pi_df_only['last_name'].apply(lambda x: x.split(' (')[0])

In [100]:
# remove names with brackets
pi_df_only['first_name'] = pi_df_only['first_name'].apply(lambda x: x.split(' (')[0])
pi_df_only['last_name'] = pi_df_only['last_name'].apply(lambda x: x.split(' (')[0])

In [117]:
pis_scopid_dict = {}
for index, row in tqdm(pi_df_only.iterrows(), total=pi_df_only.shape[0]): #stopped at 22901
    scopid = -1
    try:
        if row['organization.org_country'] != '':
            s = AuthorSearch(f"AUTHLAST({row['last_name']}) and AUTHFIRST({row['first_name']}) and AFFILCOUNTRY({row['organization.org_country']})")
        else:
            s = AuthorSearch(f"AUTHLAST({row['last_name']}) and AUTHFIRST({row['first_name']})")

        tmp_df = pd.DataFrame(s.authors).fillna('')
        if len(tmp_df)<1:
            pis_scopid_dict[row['contact_pi_name']] = scopid
            continue
        # try first case:
        if tmp_df['affiliation'].str.contains(row['organization.org_name'],case=False,regex=False).sum() > 0:
            scopid = tmp_df[tmp_df['affiliation'].str.contains(row['organization.org_name'],case=False,regex=False)].sort_values(by='documents',ascending = False)['eid'].iloc[0].split('-')[-1]
            pis_scopid_dict[row['contact_pi_name']] = scopid
            pi_df_only.loc[index,'case_caught'] = 'FIRST_CASE'
            pi_df_only.to_excel("pi_df_only_caught.xlsx",index=False)
            continue
        # try second case:
        if tmp_df['affiliation'].str.contains(row['org_clean'],case=False,regex=False).sum() > 0:
            scopid = tmp_df[tmp_df['affiliation'].str.contains(row['org_clean'],case=False,regex=False)].sort_values(by='documents',ascending = False)['eid'].iloc[0].split('-')[-1]
            pis_scopid_dict[row['contact_pi_name']] = scopid
            pi_df_only.loc[index,'case_caught']  = 'SECOND_CASE'
            pi_df_only.to_excel("pi_df_only_caught.xlsx",index=False)
            continue
        # try third case:
        if tmp_df['city'].str.contains(row['organization.org_city'],case=False,regex=False).sum() > 0:
            scopid = tmp_df[tmp_df['city'].str.contains(row['organization.org_city'],case=False,regex=False)].sort_values(by='documents',ascending = False)['eid'].iloc[0].split('-')[-1]
            pis_scopid_dict[row['contact_pi_name']] = scopid
            pi_df_only.loc[index,'case_caught']  = 'THIRD_CASE'
            pi_df_only.to_excel("pi_df_only_caught.xlsx",index=False)
            continue
        pis_scopid_dict[row['contact_pi_name']] = scopid
    except Exception as ae: 
        if isinstance(ae, ScopusQueryError):
            pis_scopid_dict[row['contact_pi_name']] = scopid
            continue
        else:
            pi_df_only.to_excel("pi_df_only_caught.xlsx",index=False)
            with open('pi_scopid.pickle', 'wb') as handle:
                pickle.dump(pis_scopid_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
            raise
        
with open('pi_scopid.pickle', 'wb') as handle:
    pickle.dump(pis_scopid_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
pi_df_only.to_excel("pi_df_only_caught.xlsx",index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24964/24964 [31:35:24<00:00,  4.56s/it]


# Load the mined PI Scopus ID's

In [170]:
with open('pi_scopid.pickle', "rb") as input_file:
    pis_scopid_dict = pickle.load(input_file)

In [171]:
pi_df_only = pd.read_excel("pi_df_only_caught.xlsx")

In [172]:
pi_df_only['ScopusID'] = pi_df_only['contact_pi_name'].map(pis_scopid_dict)

## Statistics on the retrival process:

In [173]:
found_pidf = pi_df_only[~((pi_df_only['ScopusID'] == -1) | (pi_df_only['case_caught'].isna()))].reset_index(drop=True)

In [174]:
total_pis = len(pi_df_only)
missing_ids = len(pi_df_only[(pi_df_only['ScopusID'] == -1) | (pi_df_only['case_caught'].isna())])

pis_found = found_pidf['ScopusID'].shape[0]

scop_ids_found = found_pidf['ScopusID'].unique().shape[0]

duplicated_ids = (found_pidf['ScopusID'].value_counts() > 1).sum()
duplicated_ids_vals = (found_pidf['ScopusID'].value_counts()[found_pidf['ScopusID'].value_counts() > 1]).index


duplicated_names = (found_pidf['contact_pi_name'].value_counts() > 1).sum()
duplicated_names_vals = (found_pidf['contact_pi_name'].value_counts()[found_pidf['contact_pi_name'].value_counts() > 1]).index


profile_ids = (found_pidf['profile_id'].value_counts() > 1).sum()
profile_ids_vals = found_pidf['profile_id'].value_counts()[found_pidf['profile_id'].value_counts() > 1].index

In [175]:
print(f"PI Summary:\n\
- {missing_ids} Scopus IDs were not retrieved out of {total_pis:,} PIs.\n\
- {scop_ids_found:,} SCOPUS IDs were retrieved for {pis_found:,} inital PIs, indicating duplicated names.\n\
- {duplicated_ids:,} IDs appeared more than once, while {profile_ids:,} PIs profile ids were duplicated\n\
- and there were {duplicated_names} PI names that repeated more than one")
# - {duplicated_ids:,} IDs appeared more than once, while {duplicated_names:,} pi full names were duplicated and .")


PI Summary:
- 908 Scopus IDs were not retrieved out of 24,964 PIs.
- 21,658 SCOPUS IDs were retrieved for 24,056 inital PIs, indicating duplicated names.
- 2,131 IDs appeared more than once, while 1,967 PIs profile ids were duplicated
- and there were 1524 PI names that repeated more than one


**We drop duplicated PIs with the same profile id, full name and scopus id and with the same full name and scopus id but diffrent profile id** 

In [180]:
pidf_stg1 = found_pidf.drop_duplicates(subset = ['profile_id','contact_pi_name','ScopusID'])\
        .drop_duplicates(subset = ['contact_pi_name','ScopusID'])\
        .reset_index(drop=True)
len(pidf_stg1)

22362

In [181]:
total_pis = len(pi_df_only)
missing_ids = len(pi_df_only[(pi_df_only['ScopusID'] == -1) | (pi_df_only['case_caught'].isna())])

pis_found = pidf_stg1['ScopusID'].shape[0]

scop_ids_found = pidf_stg1['ScopusID'].unique().shape[0]

duplicated_ids = (pidf_stg1['ScopusID'].value_counts() > 1).sum()
duplicated_ids_vals = (pidf_stg1['ScopusID'].value_counts()[pidf_stg1['ScopusID'].value_counts() > 1]).index


duplicated_names = (pidf_stg1['contact_pi_name'].value_counts() > 1).sum()
duplicated_names_vals = (pidf_stg1['contact_pi_name'].value_counts()[pidf_stg1['contact_pi_name'].value_counts() > 1]).index


profile_ids = (pidf_stg1['profile_id'].value_counts() > 1).sum()
profile_ids_vals = pidf_stg1['profile_id'].value_counts()[pidf_stg1['profile_id'].value_counts() > 1].index

In [184]:
((pidf_stg1['case_caught'].value_counts(normalize=True) * 100).round()).astype(int).astype(str)+"%"

THIRD_CASE     49%
FIRST_CASE     48%
SECOND_CASE     3%
Name: case_caught, dtype: object

## Merge Scopus Ids with winning appilcations

In [ ]:
df_final = pd.read_csv('NIH_GrantCalls_Winners.csv')

In [216]:
df_wins_final= pd.merge(df_final,pidf_stg1[['contact_pi_name','ScopusID']],on='contact_pi_name')

In [219]:
df_wins_final.appl_id.unique().shape[0] , df_wins_final.shape[0]

(128586, 128586)

In [222]:
df_wins_final.to_csv('df_nih_wins_22362.csv',index=False)

# Get publications based on scopus ID of extracted PIs

In [2]:
df_nih = pd.read_csv('df_nih_wins_22362.csv')

c:\users\gzeevi\pycharmprojects\pythonproject\venv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,26,29,33,50,70,73) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# import pybliometrics.scopus
import pybliometrics.scopus
# pybliometrics.scopus.utils.create_config()
from pybliometrics.scopus import config, ScopusSearch, AbstractRetrieval,AuthorSearch,AuthorRetrieval,SubjectClassifications
from pybliometrics.scopus import AffiliationSearch,AffiliationRetrieval
from pybliometrics.scopus.exception import Scopus429Error

In [5]:
pubs_df = pd.DataFrame(columns = ['eid', 'doi', 'pii', 'pubmed_id', 'title', 'subtype', 'subtypeDescription',
                                  'creator', 'afid', 'affilname', 'affiliation_city', 'affiliation_country', 'author_count',
                                  'author_names', 'author_ids', 'author_afids', 'coverDate', 'coverDisplayDate',
                                  'publicationName', 'issn', 'source_id', 'eIssn', 'aggregationType', 'volume',
                                  'issueIdentifier', 'article_number', 'pageRange', 'description',
                                  'authkeywords', 'citedby_count', 'openaccess', 'fund_acr', 'fund_no', 'fund_sponsor','scopus_id','Name'])

In [6]:
df_nih_iter = df_nih[['contact_pi_name','ScopusID']].drop_duplicates().reset_index(drop=True)

In [ ]:
try:
    for index in tqdm(range(df_nih_iter.shape[0])):
        row = df_nih_iter.iloc[index]
        au = AuthorRetrieval(row['ScopusID'], refresh=True)
        tmp_df = pd.DataFrame(au.get_documents())
        tmp_df['scopus_id'] = row['ScopusID']
        tmp_df['Name'] = row['contact_pi_name']
        pubs_df = pd.concat([pubs_df, tmp_df])

    pubs_df = pubs_df.reset_index(drop=True)
    
except Exception as e:
    pubs_df.to_csv('nih_pubs.csv', index=False)
    raise e

# Save the final DataFrame to CSV
pubs_df.to_csv('nih_pubs.csv', index=False)